In [425]:
from decimal import*
beatDic = {'measure':None,
           'whole':Decimal(1),
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }
BDic = dict((v,k) for k,v in beatDic.iteritems())
print BDic

{Decimal('0.03125'): '32nd', Decimal('0.25'): 'quarter', Decimal('0.5'): 'half', Decimal('1'): 'whole', Decimal('0.00390625'): '256th', Decimal('0.015625'): '64th', None: 'measure', Decimal('0.0625'): '16th', Decimal('0.125'): 'eighth', Decimal('0.0078125'): '128th'}


In [19]:
#進mongodb
import pymongo
from pymongo import MongoClient
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  #選擇database
collect = db['tempo_beats']  #選擇database.collection
dic = {'musicname':'aaa','staff_id':2,'instrument':'flute','sig':'4/4','tempo':120,'start':1,'end':4,'beat':'0.25,0;0.5,1;0.125,1;0.125,1'}
collect.insert_one(dic) #放一筆dic

In [49]:
    #查詢mongoDB
client = MongoClient('mongodb://10.120.30.8:27017')
db = client['music']  
collect = db['tempo_beats']
cur = collect.find({'staff_id':1,'timesig':'4/4','start':1,'end':4},{'_id':0,'staff_id':1,'beats':1,'timesig':'4/4'}) 


In [394]:
# 將被合併過的拍子還原成符合四小節的拍子!，ex (3,0)=(1:0)、(1:0)、(1:0);(1.7:1) =(1:1)、(0.7:1)
def splitM(Mli,bag):
    if sum(bag) == LenMea:
        A = bag.pop()
        Mli.append({A:int(p)})
        return Mli
    elif sum(bag) > LenMea:
        least = sum(bag)-LenMea
        A = bag.pop()
        Nk = A-least
        Mli.append({Nk:int(p)})
        bag = []
        bag.append(least)
        return splitM(Mli,bag)
    elif sum(bag) < LenMea :
        Mli.append({bag[-1]:int(p)})
        return Mli


In [490]:
#Mdic = cur[0]
#test try
Mdic = {u'staff_id': 1, u'timesig': u'4/4', u'beats': u'3,0;0.1875,1;0.0625,1;0.125,0;0.125,1;0.0625,0;0.0625,1;0.0625,1;0.0625,1;0.25,1'}

N , D = Mdic['timesig'].split('/')
N=int(N);D=int(D);Ts = N/D

LenMea =Ts*128                        #每小節的長度乘上128
Mli=[];bag=[]                         # bag=[] for def slpitM()
for beat in  Mdic['beats'].split(';'):
    b , p =beat.split(',') ; Bbeat = Decimal(b)*128
    bag.append(Bbeat)
    splitM(Mli,bag)
# print Mli
bag=[];Measure=[[],[],[],[]];i=0
for s in Mli:
    bag.append(s.keys()[0])
    Measure[i].append(s)
    if sum(bag) == LenMea:
        i+=1
        bag=[]
print Measure
        

[[{Decimal('128'): 0}], [{Decimal('128'): 0}], [{Decimal('128'): 0}], [{Decimal('24.0000'): 1}, {Decimal('8.0000'): 1}, {Decimal('16.000'): 0}, {Decimal('16.000'): 1}, {Decimal('8.0000'): 0}, {Decimal('8.0000'): 1}, {Decimal('8.0000'): 1}, {Decimal('8.0000'): 1}, {Decimal('32.00'): 1}]]


In [494]:
#change type
addB =[[],[],[],[]]
for i in range(0,4):
    AA = Measure[i]
    for Bitem in range(0,len(AA)):
        Rbeat = AA[Bitem].keys()[0]
        R = Rbeat / 128
#         print R
        if R in BDic:
            addB[i].append((AA[Bitem][Rbeat],BDic[R]))
        else:
            addB[i].append((AA[Bitem][Rbeat],R))
print addB
    

[[(0, 'whole')], [(0, 'whole')], [(0, 'whole')], [(1, Decimal('0.1875')), (1, '16th'), (0, 'eighth'), (1, 'eighth'), (0, '16th'), (1, '16th'), (1, '16th'), (1, '16th'), (1, 'quarter')]]
